In [1]:
import pandas as pd
from io import StringIO
from google.colab import drive
import string
import numpy


drive.mount('/content/drive')
uk_path ='/content/drive/My Drive/UK/test_for_validation.csv'
spain_path = '/content/drive/My Drive/Spain/development/test_indiv_results.csv'
france_path = '/content/drive/My Drive/France/development/test_indiv_results.csv'
italy_path = '/content/drive/My Drive/Italy/development/test_indiv_results.csv'
germany_path ='/content/drive/My Drive/Germany/development/test_indiv_results.csv'

df_uk = pd.read_csv(germany_path, index_col=0, engine='python', error_bad_lines=False)
df_uk
labels = []
for i in df_uk['Label'].values.tolist():
  if i == 'None' or i == None:
    labels.append(0)
  else:
    labels.append(1)
print(len(labels))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
592765


# Random Weighted and Uniform Classifiers

In [11]:
unique, unique_counts = numpy.unique(numpy.array(labels), return_counts=True)
df_uk['predictions'].values

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
random_dist = numpy.random.randint(0,2, sum(unique_counts))
random_dist

array([1, 1, 0, ..., 1, 0, 1])

In [13]:
weighted = numpy.array([unique[0]] * unique_counts[0] + [unique[1]] * unique_counts[1])
numpy.random.shuffle(weighted)

# Chi-squared

In [4]:
from scipy.stats import chisquare
def pred_type(variable, truth):
  tp, fp, tn, fn = 0, 0, 0, 0
  for v, t in zip(variable, truth):
    if v==1 and t==1:
      tp+=1
    elif v==1 and t==0:
      fp+=1
    elif v==0 and t==0:
      tn+=1
    elif v==0 and t==1:
      fn+=1
  return numpy.array([tp+tn, fp+fn])
  #return numpy.array([tp, fp, tn, fn])

class_types = pred_type(df_uk['predictions'].values, labels)
random_types =pred_type(random_dist, labels)
print(class_types)
print(random_types)

ch, p = chisquare(random_types, f_exp = class_types)
print('Class - Random')
print(ch, p)

print('---------------')
print(unique_counts)



random_weighted_type = pred_type(weighted, labels)
print(sum(random_weighted_type), sum(class_types))
ch, p = chisquare(random_weighted_type, f_exp=class_types)
print('Class - Weighted Random')
print(ch, p)


[547394  45371]
[296042 296723]
Class - Random
1507887.214932009 0.0
---------------
[560698  32067]
592765 592765
Class - Weighted Random
5686.138529781042 0.0


# Fine-grained approach (True Positives, False Positives, True Negatives, False Negatives)

In [9]:
def pred_type(predictions, truth):
  p_type= []
  for p, gt in zip(predictions, truth):
    if p==1 and gt==1:
      p_type.append('TP')
    elif p==1 and gt==0:
      p_type.append('FP')
    elif p==0 and gt==1:
      p_type.append('FN')
    elif p==0 and gt==0:
      p_type.append('TN')
  return p_type


def pre_type_overlap(classifier, benchmark, labels):
  tp_tp, fp_tp, tn_tp, fn_tp = 0, 0, 0, 0
  tp_fp, fp_fp, tn_fp, fn_fp = 0, 0, 0, 0
  tp_tn, fp_tn, tn_tn, fn_tn =0, 0, 0, 0
  tp_fn, fp_fn, tn_fn, fn_fn =0, 0, 0, 0
  class_type = pred_type(classifier, labels)
  bench_type = pred_type(benchmark, labels)
  print(set(class_type))
  print(set(bench_type))
  for c, b in zip(class_type, bench_type):
    if b =='TP':
      if c =='TP':
        tp_tp+=1
      elif c =='FP':
        fp_tp +=1
      elif c =='TN':
        tn_tp+=1
      elif c =='FN':
        fn_tp+=1

    elif b =='FP':
      if c =='TP':
        tp_fp+=1
      elif c =='FP':
        fp_fp +=1
      elif c =='TN':
        tn_fp+=1
      elif c =='FN':
        fn_fp+=1
    
    elif b == 'TN':
      if c =='TP':
        tp_tn+=1
      elif c =='FP':
        fp_tn +=1
      elif c =='TN':
        tn_tn+=1
      elif c =='FN':
        fn_tn+=1
    
    elif b == 'FN':
      if c =='TP':
        tp_fn+=1
      elif c =='FP':
        fp_fn +=1
      elif c =='TN':
        tn_fn+=1
      elif c =='FN':
        fn_fn+=1
  
  return numpy.array([ numpy.array([tp_tp, fp_tp, tn_tp, fn_tp]), 
                      numpy.array([tp_fp, fp_fp, tn_fp, fn_fp]), 
                      numpy.array([tp_tn, fp_tn, tn_tn, fn_tn]), 
                      numpy.array([tp_fn, fp_fn, tn_fn, fn_fn])])
  
#cont_table_detail = pre_type_overlap(df_uk['predictions'].values, random_dist, labels)
#print(cont_table_detail)

class_type = cont_table_detail.sum(axis=0)
bench_type = cont_table_detail.sum(axis=1)
cont_table_detail_type = numpy.array([class_type,bench_type])


chi2, p = chisquare(cont_table_detail_type[1], f_exp=cont_table_detail_type[0])
print(chi2, p)


print(cont_table_detail_type, type(cont_table_detail_type))
#chi2, p = chi2_contingency(list(cont_table_detail_type))
#chi2, p, dof, ex = chisquare(cont_table_detail)
print(chi2, p)

2941022.039669541 0.0
[[ 10196  23500 537198  21871]
 [ 15962 280618 280080  16105]] <class 'numpy.ndarray'>
2941022.039669541 0.0
